In [2]:
'''This script is going to perform xgboost regression on the training set created using "scheme 2" 
(see other documentation).'''

import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import OneHotEncoder, TargetEncoder
from sklearn.metrics import (
    mean_squared_error, mean_absolute_error, r2_score,
    root_mean_squared_log_error
)
from sklearn.model_selection import GridSearchCV
from utils import (
    describe_dataframe, df_train_test, graph_results, reg_train_eval
)

In [3]:
'''Data processing and feature engineering. This is done manually every time so that I can maintain customizability.'''

main = pd.read_csv("../data/combo_data.csv")
main

#Identify relevant columns
select_cols = ["LOS" ,
               "YR",  
               "PRNCPAL_DGNS_CD", 
               "CLM_IP_ADMSN_TYPE_CD", 
               "ER_flag", 
               "STATE_CODE", 
               "COUNTY_CD", 
               "BENE_RACE_CD", 
               "ESRD_IND",
               "Age", 
               "TOT_RX_CST_AMT", 
               "NUM_DIAG",
               "SEX_IDENT_CD"]

workingdf_te = main[select_cols].copy()

prncpl_diag_col = pd.DataFrame(data = {"PRNCPL_DGNS_CD": workingdf_te.loc[:,"PRNCPAL_DGNS_CD"]})

print(prncpl_diag_col.shape)

los_col = workingdf_te.loc[:,"LOS"]

print(los_col.shape)

encoder = TargetEncoder(categories='auto', target_type='continuous', smooth='auto', cv=5, random_state=42)

workingdf_te["PRNCPAL_DGNS_CD"] = encoder.fit_transform(prncpl_diag_col, los_col)

workingdf_te = workingdf_te.assign(ESRD_IND = workingdf_te["ESRD_IND"].map({"Y": 1, "0" : 0}))

wdf_rest_te = workingdf_te[workingdf_te["YR"] < 2022]
wdf_2022_te = workingdf_te[workingdf_te["YR"] >= 2022]

ohe = OneHotEncoder(sparse_output=False)

ohe.fit(wdf_rest_te[['CLM_IP_ADMSN_TYPE_CD', 
                  'STATE_CODE', 
                  'BENE_RACE_CD', 
                  "SEX_IDENT_CD"]])

ohe_df_rest_te = pd.DataFrame(data = ohe.transform(wdf_rest_te[['CLM_IP_ADMSN_TYPE_CD', 
                                                          'STATE_CODE', 
                                                          'BENE_RACE_CD',
                                                          "SEX_IDENT_CD"]]), 
             columns=ohe.get_feature_names_out(wdf_rest_te[['CLM_IP_ADMSN_TYPE_CD', 
                                                         'STATE_CODE', 
                                                         'BENE_RACE_CD',
                                                         "SEX_IDENT_CD"]].columns))

ohe.fit(wdf_2022_te[['CLM_IP_ADMSN_TYPE_CD', 
                  'STATE_CODE', 
                  'BENE_RACE_CD', 
                  "SEX_IDENT_CD"]])
ohe_df_2022_te = pd.DataFrame(data = ohe.transform(wdf_2022_te[['CLM_IP_ADMSN_TYPE_CD', 
                                                          'STATE_CODE', 
                                                          'BENE_RACE_CD',
                                                          "SEX_IDENT_CD"]]), 
             columns=ohe.get_feature_names_out(wdf_2022_te[['CLM_IP_ADMSN_TYPE_CD', 
                                                         'STATE_CODE', 
                                                         'BENE_RACE_CD',
                                                         "SEX_IDENT_CD"]].columns))

#drop year, county code, all one hot encoded vars 
wdf_rest_te = wdf_rest_te.drop(columns=["YR", 
                                  "COUNTY_CD", 
                                  'CLM_IP_ADMSN_TYPE_CD', 
                                  'STATE_CODE', 
                                  'BENE_RACE_CD',
                                  "SEX_IDENT_CD"])
wdf_2022_te = wdf_2022_te.drop(columns=["YR", 
                                  "COUNTY_CD", 
                                  'CLM_IP_ADMSN_TYPE_CD', 
                                  'STATE_CODE', 
                                  'BENE_RACE_CD',
                                  "SEX_IDENT_CD"])

wdf_rest_te = pd.concat([wdf_rest_te.reset_index(drop=True), ohe_df_rest_te.reset_index(drop=True)], axis=1)
wdf_2022_te = pd.concat([wdf_2022_te.reset_index(drop=True), ohe_df_2022_te.reset_index(drop=True)], axis=1)

reg_mod_metrics = {"Test":{},
                   "Train":{}}

predictions = {}

X_train_rest_te, X_test_rest_te, y_train_rest_te, y_test_rest_te = df_train_test(wdf_rest_te, "LOS", 0.2)

X_train_2022_te, X_test_2022_te, y_train_2022_te, y_test_2022_te = df_train_test(wdf_2022_te, "LOS", 0.2)

(20867, 1)
(20867,)


In [4]:
'''Transform pandas dataframes into xgboost DMMatrix objects.'''

dtrain_rest_te = xgb.DMatrix(X_train_rest_te, label=y_train_rest_te)
dtest_rest_te = xgb.DMatrix(X_test_rest_te, label=y_test_rest_te)
dtrain_2022_te = xgb.DMatrix(X_train_2022_te, label=y_train_2022_te)
dtest_2022_te = xgb.DMatrix(X_test_2022_te, label=y_test_2022_te)

#params
# default objective function: squared error
params = {"max_depth": 5, "eval_metric":"rmsle"}

#evals (specifies which sets are training and test)
watchlist = [(dtest_rest_te, "eval"), (dtrain_rest_te, "train")]

num_round = 2
bst = xgb.train(params, dtrain_rest_te, num_boost_round=num_round, evals=watchlist)

preds = bst.predict(dtest_2022_te).round()
labels = dtest_2022_te.get_label()

rmsle_test = root_mean_squared_log_error(labels, preds)
r2_test = r2_score(labels, preds)
mae_test = mean_absolute_error(labels, preds)
mse_test = mean_squared_error(labels, preds)

print(f"Test RMSLE: {rmsle_test}")
print(f"Test R2: {r2_test}")
print(f"Test MAE: {mae_test}")
print(f"Test MSE: {mse_test}")



[0]	eval-rmsle:0.57247	train-rmsle:0.56399
[1]	eval-rmsle:0.45720	train-rmsle:0.44785
Test RMSLE: 0.6776822805404663
Test R2: 0.28062623739242554
Test MAE: 1.4815725088119507
Test MSE: 9.506142616271973


In [5]:
from sklearn.model_selection import RandomizedSearchCV
import scipy.stats as stats

param_dist = {
    'max_depth': stats.randint(3, 10),
    'learning_rate': stats.uniform(0.01, 0.3),
    'subsample': stats.uniform(0.5, 0.5),
    'n_estimators': stats.randint(50, 300)
}

# XGBoost model with squared error loss (default regression)
xgb_model = xgb.XGBRegressor(objective='reg:squarederror', random_state=42)

# Set up RandomizedSearchCV
random_search = RandomizedSearchCV(
    xgb_model, 
    param_distributions=param_dist, 
    n_iter=20, 
    cv=5, 
    scoring='r2'
)

# Fit to training data
random_search.fit(X_train_rest_te, y_train_rest_te)

print("Best hyperparameters:", random_search.best_params_)
print("Best score:", random_search.best_score_)

# Make predictions on test set
y_pred = random_search.best_estimator_.predict(X_test_rest_te)

# Clip negative predictions to 0
y_pred = np.clip(y_pred, 0, None)

# Now you can evaluate performance or output predictions
print(y_pred)

Best hyperparameters: {'learning_rate': 0.045354822141755805, 'max_depth': 4, 'n_estimators': 51, 'subsample': 0.6730326080921576}
Best score: 0.533855390548706
[0.09913564 0.09913564 2.7148943  ... 0.09913564 0.09913564 0.23162922]


In [6]:
#random_search.best_params_

best_params = {'learning_rate': 0.04928091996137682,
 'max_depth': 4,
 'n_estimators': 111,
 'subsample': 0.6643614397837909}

In [7]:
best_model = xgb.XGBRegressor(objective='reg:squarederror', random_state=42, learning_rate = 0.04928091996137682,
                               max_depth = 4,
                               n_estimators = 111,
                               subsample = 0.6643614397837909)
reg_train_eval(best_model,
               None,
               X_train_rest_te,
               y_train_rest_te,
               X_test_2022_te,
               y_test_2022_te,
               reg_mod_metrics,
               predictions,
               year = "rest_train-2022test-te-gridcv_best")

reg_mod_metrics['Test']

{'XGBRegressor - NoneType - rest_train-2022test-te-gridcv_best': {'RMSLE': 0.30969713047142367,
  'R2': 0.4739733338356018,
  'MAE': 0.7689482860761427,
  'MSE': 6.951163206929393}}

In [8]:
# Target Encoding Diagnosis Codes

workingdf_te = main[select_cols].copy()

prncpl_diag_col = pd.DataFrame(data = {"PRNCPL_DGNS_CD": workingdf_te.loc[:,"PRNCPAL_DGNS_CD"]})

print(prncpl_diag_col.shape)

los_col = workingdf_te.loc[:,"LOS"]

print(los_col.shape)

encoder = TargetEncoder(categories='auto', target_type='continuous', smooth='auto', cv=5, random_state=42)

workingdf_te["PRNCPAL_DGNS_CD"] = encoder.fit_transform(prncpl_diag_col, los_col)

workingdf_te

workingdf_te = workingdf_te.assign(ESRD_IND = workingdf_te["ESRD_IND"].map({"Y": 1, "0" : 0}))

wdf_rest_te = workingdf_te[workingdf_te["YR"] < 2022]
wdf_2023_te = workingdf_te[workingdf_te["YR"] >= 2022]

ohe = OneHotEncoder(sparse_output=False)

ohe.fit(wdf_rest_te[['CLM_IP_ADMSN_TYPE_CD', 
                  'STATE_CODE', 
                  'BENE_RACE_CD', 
                  "SEX_IDENT_CD"]])

ohe_df_rest_te = pd.DataFrame(data = ohe.transform(wdf_rest_te[['CLM_IP_ADMSN_TYPE_CD', 
                                                          'STATE_CODE', 
                                                          'BENE_RACE_CD',
                                                          "SEX_IDENT_CD"]]), 
             columns=ohe.get_feature_names_out(wdf_rest_te[['CLM_IP_ADMSN_TYPE_CD', 
                                                         'STATE_CODE', 
                                                         'BENE_RACE_CD',
                                                         "SEX_IDENT_CD"]].columns))

ohe.fit(wdf_2023_te[['CLM_IP_ADMSN_TYPE_CD', 
                  'STATE_CODE', 
                  'BENE_RACE_CD', 
                  "SEX_IDENT_CD"]])
ohe_df_2023_te = pd.DataFrame(data = ohe.transform(wdf_2023_te[['CLM_IP_ADMSN_TYPE_CD', 
                                                          'STATE_CODE', 
                                                          'BENE_RACE_CD',
                                                          "SEX_IDENT_CD"]]), 
             columns=ohe.get_feature_names_out(wdf_2023_te[['CLM_IP_ADMSN_TYPE_CD', 
                                                         'STATE_CODE', 
                                                         'BENE_RACE_CD',
                                                         "SEX_IDENT_CD"]].columns))

#drop year, county code, all one hot encoded vars 
wdf_rest_te = wdf_rest_te.drop(columns=["YR", 
                                  "COUNTY_CD", 
                                  'CLM_IP_ADMSN_TYPE_CD', 
                                  'STATE_CODE', 
                                  'BENE_RACE_CD',
                                  "SEX_IDENT_CD"])
wdf_2023_te = wdf_2023_te.drop(columns=["YR", 
                                  "COUNTY_CD", 
                                  'CLM_IP_ADMSN_TYPE_CD', 
                                  'STATE_CODE', 
                                  'BENE_RACE_CD',
                                  "SEX_IDENT_CD"])

wdf_rest_te = pd.concat([wdf_rest_te.reset_index(drop=True), ohe_df_rest_te.reset_index(drop=True)], axis=1)
wdf_2023_te = pd.concat([wdf_2023_te.reset_index(drop=True), ohe_df_2023_te.reset_index(drop=True)], axis=1)

(20867, 1)
(20867,)


In [9]:
X_train_rest_te, X_test_rest_te, y_train_rest_te, y_test_rest_te = df_train_test(wdf_rest_te, "LOS", 0.2)

X_train_2022_te, X_test_2022_te, y_train_2022_te, y_test_2022_te = df_train_test(wdf_2023_te, "LOS", 0.2)

In [10]:
from sklearn.model_selection import RandomizedSearchCV
import scipy.stats as stats

param_dist = {
    'max_depth': stats.randint(3, 10),
    'learning_rate': stats.uniform(0.01, 0.3),
    'subsample': stats.uniform(0.5, 0.5),
    'n_estimators': stats.randint(50, 300)
}

# XGBoost model with squared error loss (default regression)
xgb_model = xgb.XGBRegressor(objective='reg:squarederror', random_state=42)

# Set up RandomizedSearchCV
random_search = RandomizedSearchCV(
    xgb_model, 
    param_distributions=param_dist, 
    n_iter=20, 
    cv=5, 
    scoring='r2'
)

# Fit to training data
random_search.fit(X_train_rest_te, y_train_rest_te)

print("Best hyperparameters:", random_search.best_params_)
print("Best score:", random_search.best_score_)

# Make predictions on test set
y_pred = random_search.best_estimator_.predict(X_test_rest_te)

# Clip negative predictions to 0
y_pred = np.clip(y_pred, 0, None)

# Now you can evaluate performance or output predictions
print(y_pred)

Best hyperparameters: {'learning_rate': 0.017079211753633243, 'max_depth': 6, 'n_estimators': 293, 'subsample': 0.7616578158156807}
Best score: 0.511291241645813
[0.00636277 0.01490354 3.372079   ... 0.00554755 0.00836424 0.6275119 ]


In [11]:
random_search.best_params_

{'learning_rate': 0.017079211753633243,
 'max_depth': 6,
 'n_estimators': 293,
 'subsample': 0.7616578158156807}

In [ ]:
best_model = xgb.XGBRegressor(objective='reg:squarederror', random_state=42, learning_rate = 0.017079211753633243,
                               max_depth = 6,
                               n_estimators = 293,
                               subsample = 0.7616578158156807)
reg_train_eval(best_model,
               None,
               X_train_rest_te,
               y_train_rest_te,
               X_test_2022_te,
               y_test_2022_te,
               reg_mod_metrics,
               predictions,
               year = "rest_train-2022test-te-gridcv_best")

reg_mod_metrics['Test']

{'XGBRegressor - NoneType - rest_train-2022test-te-gridcv_best': {'RMSLE': 0.3015823299857391,
  'R2': 0.4891522526741028,
  'MAE': 0.7533036715323014,
  'MSE': 6.7505823669407015}}